<a href="https://colab.research.google.com/github/Benned-H/Summer2019/blob/master/Speech%20and%20Language%20Processing/Chapter_2_Regular_Expressions_and_Automata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regular Expressions and Automata (p. 20)

**Regular expression** - The standard notation for characterizing text sequences.

**Finite-state automaton** - The mathematical device used to implement RegEx as well as the basis for variants such as the finite-state transducer, Hidden Markov Models, and *N*-gram grammars.

## 2.1 Regular Expressions (p. 21)

**Regular expression** (RE) is the standard for searching texts in UNIX, Microsoft Word, and various web search engines. It's used to writes formula for specifying certain classes of strings. **Strings** are sequences of symbols, typically alphanumeric. A space is a character like any other, represented by ␣.

Regular expressions are an algebraic notation for characterizing sets of strings; it thus has operands and operators. Only three operators are necessary, but we'll use the syntax of the Perl language because it's more convenient. A RE search needs a *pattern* and a *corpus* of texts to search through. Depending on the medium, a search might return entire documents, web pages, single words, or entire lines of text. ```grep``` does this last one.

### Basic RegEx Patterns

The simplest search is a sequence of characters. We type ```/text/``` to return all instances of the substring *text* from our corpus. These slashes are Perl notation, not part of the regular expression. RegEx is *case sensitive*, so ```/m/``` will return different results than ```/M/```. We could use square braces to specify a disjunction: ```/[mM]/``` matches patterns containing *m* or *M*.

Extending this disjunction, we can specify any single digit using ```/[0123456789]/```. But this gets awkward for the entire alphabet, so we introduce the **dash** to specify any character in a range, e.g. ```/[0-4]/``` matches any of *0*, *1*, *2*, *3*, or *4*. Also, ```/[A-Z]/``` matches an uppercase letter, ```/[a-z]/``` matches any lowercase letter, or ```/[0-9]/``` any single digit.

Also inside square brackets, the **caret** can negate what a single character can be. If the carat is the first symbol after the open bracket ```[```, the pattern is negated. Thus ```/[^a]/``` matches any character except *a*, but ```/[a^]/``` matches *a* or *^*. We can use the **question-mark** to say 'the preceding character or nothing,' e.g. ```/colou?r/``` matches *color* or *colour*.

As for zero or more of some pattern, we can use the **Kleene star**, \* (also closure). For example, ```/0*/``` will match any string with zero or more 0s. More complex patterns can be repeated, where ```/[me]*/``` will match "zero or more *m*s or *e*s," like *mmmm* or *meme* or *memeemem*. As added help, sometimes we want one or more of some pattern, and we can use the **Kleene +** for this. Thus ```/[0-9]+/``` means any sequence of digits.

The **wildcard** character is the period (```/./```). This matches any single character except a carriage return (newline). This can be used to find any string of characters when combined with the Kleene star, e.g. ```/apple.*apple/``` will find any lines where the word *apple* appears twice.

**Anchors** are characters that anchor regular expressions to certain places in a string. The most common are the caret, ```/^/```, and dollar sign. The caret matches the start of a line, so ```/^The/``` will match lines starting with the word *The*. Similarly, a dollar sign matches the end of a line. We can use a backslash to make a ```/./``` mean 'period' and not the wildcard.

Two other anchors are ```\b```, which matches word boundaries, and ```\B```, which matches non-boundaries. Thus ```/\bbro\b/``` matches the word *bro*, but not *brother*. Perl defines words as any sequence of digits, underscores, or letters. Thus ```/\b99\b/``` matches *99* or *\$99*.

### Disjunction, Grouping, and Precedence (p. 25)

We need a way to search for two alternative words, which the **disjunction** (also union) operator ```|``` does. ```/cat|dog/``` matches *cat* or *dog*. We can use the **parentheses** operator to group strings with a higher precedence than union. This allows us to creates searches like ```/pupp(y|ies)/```, so that disjunction applies to only the suffixes. We can also use ```()```s to apply closure on an entire string.

Now that we've introduced some precedence into our operators, we should fully define this **operator precedence hierarchy**:
1. Parenthesis: ```/()/```
2. Counters: ```/* + ? {}/```
3. Sequences and anchors ```/the ^ $/```
4. Disjunction: ```/|/```

We also find ambiguity in how patterns might match strings: consider ```/[a-z]*/``` on *once upon a time*. Will it match zero, one, all the letters? We define RegEx to match the largest expressions it can; patterns are greedy, expanding to cover as much of a string as they can.

### A Simple Example

Suppose we want to find instances of the word *the*. Consider the following evolutions of our query:
1. ```/the/```. But capitalization!
2. ```/[Tt]he/```. Yet what about occurances inside words? Avoid *theology*.
3. ```/\b[Tt]he\b/```. Perhaps, though, we want to find *the* in contexts next to numbers or underscores. We could instead specify:
4. ```/[^a-zA-Z][Tt]he[^a-zA-Z]```. But this requires some character on either side of the word. Finally, we have:
5. ```/(^|[^a-zA-Z])[Tt]he($|[^a-zA-Z])/```.

This process highlights two important considerations in speech and language processing:
1. We want to minimize **false positives**, like matching *other* or *there*. By avoiding incorrect matches, we increase **accuracy**.
2. We want to minimize **false negatives**, like missing *The*. By catching these cases we increase **coverage**.

### A More Complex Example

This section considers regular expressions helping someone purchase a new computer online. I won't write it fully out, but highlights:
* Use ```/\b$[0-9]+(\.[0-9][0-9])?\b/``` to find prices.
* Use ```/```␣```*/``` for zero or more spaces.

### Advanced Operators

Additional options are included in Perl notation:
* ```\d``` denotes any digit.
* ```\D``` denotes any non-digit.
* ```\w``` denotes any alphanumeric or underscore.
* ```\W``` denotes ```[^\w]```.
* ```\s``` denotes whitespace such as a space or tab.
* ```\S``` denotes ```[^\s]```.
* ```\n``` is a newline.
* ```\t``` is a tab.

We can also use numbers in brackets to indicate a certain number of the previous expression, e.g. ```/Ben{2}ed/``` will match my name, with exactly two *n*s. A range can be specified, so ```/{n,m}/``` specifies from *n* to *m* occurances of the previous expression, or ```/{n,}/``` means at least *n* copies. Thus:

* ```/*/``` acts like ```/{0,}/```.
* ```/+/``` acts like ```/{1,}/```.
* ```/?/``` acts like ```/{0,1}/```.

### Regular Expression Substitution, Memory, and ELIZA

The Perl substitution operator lets a string characterized by some RegEx to be replaced by another string: ```s/colour/color/```. If we want to refer back to the subpart matching the first pattern, we can put parentheses around the first pattern and use the **number** operator  ```\1``` to refer back. Thus ```s/([0-9]+)/<\1>/``` puts angle brackets around all numbers.

We can also use this notation to require that some string appear twice in our pattern, e.g. ```/the (.*)er they were, the \1er they will be/``` requires that the ```\1``` substring be the same thing as the ```(.*)```. A second set of parentheses could be used and then called back to using ```\2```. These numbered memories are called **registers** and are not necessarily a part of all RegEx languages.

This gives us enough tools to understand ELIZA, which just used a cascade of these kinds of substitutions which matched and then changed input lines. The first substitutions changed *my* to *your*, *I'm* to *you're*, and so on. Next, specific phrases or words were targeted and either reused using number operators or given some standard response phrase. These later substitutions were given ranks and applied in order to avoid issues.

## 2.2 Finite-State Automata (p. 30)

Regular expressions are just one way to describe **finite-state automata** (FSA). FSA are the theoretical foundation for a lot of this book, and any non-memory RegEx can be implemented using an FSA. Regular expressions also characterize a kind of formal langauge called a **regular language**. A third equivalent method, the **regular grammar**, will be discussed in Ch. 15.

### Using an FSA to Recognize Sheeptalk

Let's say we want to recognize some language for sheep:
* $L(sheep)=$ ```/baa+!/```.

We want to create an automaton (also **finite automaton**, **finite-state automaton**, or **FSA**) that recognizes the set of strings in this langauge. The FSA is a directed graph:
* A finite set of nodes representing **states**.
* States can be connected by edges called **transitions**.

Any FSA will also need:
* A **start state**, where the automaton initially begins. Denote this with an incoming arrow.
* A final or **accepting state**, which we denote with a double circle.

To see if the FSA recognizes, or **accepts** a given string, think of the input characters as a tape broken into cells. We start the FSA in its initial state and repeat this process:
1. Check the next input symbol.
2. If it matches a transition leaving the current state, move to that next state and advance one symbol on the input. Else we **reject** the input.
3. At the end of the input: If we're in an accepting state, accept. Otherwise reject.

We can represent the FSA with a **state-transition** table, where the rows are states and columns the possible input characters. Even more formally, we can define a FSA as follows:

FSA M is a 5-tuple $<Q, \sum, F, \delta(q,i), q_0>$, where:
* $Q=\{q_0, q_1, ... , q_{N-1}\}$ (set of states)
* $\sum=\{s_0, s_1, s_2, ...\}$ (finite set of input symbols)
* $F=\{q_i, q_j, q_k...\}$ (set of accepting states, $F\subseteq Q$)
* $\delta(q,i)=$ transition function (write as a state-transition table, $\delta: Q\times \sum \implies Q$)
* $q_0=$ start state.

If a FSA has only one option for all states on all characters, then we call it a **deterministic finite automaton**, or **DFA**. If there are choices, though, it's a **nondeterministic finite automaton**, or **NFA**. Pseudocode for running a DFA is included in the pdf on page 33. A final addition is the idea of a **fail state**, which the automaton transitions to on any invalid input.

### Formal Languages



1.   List item
2.   List item



A **formal language** is defined as "a model which can both generate and recognize all and only the strings of a formal language acts as a definition of the formal langauge." A formal language is a set of strings composed of symbols from some finite symbol-set called an **alphabet**. Our sheep example has alphabet $\sum=\{a,b,!\}$. Given some model $m$, we can define language $L(m)$ as the formal language characterized by $m$. Note that formal languages are not **natural languages**, which are what humans speak. In linguistics, **generative grammar** is sometimes used to mean a grammar of a formal language.

### Another Example

We could define our input alphabet to be words instead. Perhaps we want to model the subset of English words defining numbers 1-99. We only need three states, with a transition on 1-19 from $q_0$ to $q_2$ and intermediate transitions on 20, 30, ... , 90 and 1-9. Now just add *cents* and *dollars* and we'd be dealing with money. We'd need to account for plural as well as hundreds or thousands, but it's definitely doable.

### Non-Deterministic FSAs (I call these NFAs)

I defined these above, as I've studied this topic before. We can add non-determinism by introduced $\epsilon$**-transitions**, which allow a FSA to transition on *no input*. As we parse some string using one of these automata, we need to know which arcs to follow when we have a choice. This non-determinism will come up a lot, and we have three standard **solutions to non-determinism**:
1. **Backup** - When we have a choice, put a *marker* to mark where we were in the input, and what state the automaton was in. Then if we took the wrong path, double back and try another path.
2. **Look-ahead** - We look at the next symbol in the input to see if we can decide which path to take.
3. **Parallelism** - We look at every alternative path in parallel whenever we have a choice.

At this point, we'll look into backup. Using this method, we can choose terrible paths and still remain sure that we'll be able to return to unexplored paths. This requires two things:
1. We need to remember all the alternatives for each choice point.
2. We need to store sufficient information so that we can return to any one alternative.

When we reach an endpoint (no more input or invalid input), we can double back to a previous choice point and select an unexplored alternative. For an NDA, we only need to store the state that we could have gone to and the correct position in the input. Call this combination the **search-state** of this algorithm. To avoid confusion, we'll call states of the FSA **machine-states**.

A few notes to make this algorithm work:
* We should add an $\epsilon$-column to the transition table, to account for this possibility.
* We also let each cell consist of a list of possible destination nodes for some input symbol.

The code for a backup algorithm using NDAs is then given. We track all unexplored choices using a variable ```agenda```, which is a queue (or some other DS) of search states (tuples with a machine-state and a position in the input). We initialize ```agenda``` to contain the initial state and then loop:
1. Check if the search state accepts (at the end of the input, in an accepting machine-state).
2. If not, generate the list of new possible steps and append to ```agenda```.
3. Try to retrieve the next search state from ```agenda```. If empty, reject.

Pseudocode for ```ND-Recognize``` is on pg. 39 of the pdf.

A conceptual note: We only reject at the end of the input (EOI) when dealing with DFAs. This is because an EOI in an NFA simply means "this was not the right path," as we could have other options that result in  an accepting state.

### Recognition as Search

Algorithms such as this which systematically search for solutions are known as **state-space search** algorithms. Such a problem definition creates a space of possible solutions which we need to explore in some way. In ```ND-Recognize``` this space was the pairs of machine-states and input positions. But in what order should we check these states?

We want to avoid checking unfruitful states. Two potential options could be:
1. Check recently created states first (LIFO). Such a **depth-first search** implementation could use a **stack**. The downfall of this option is that we can enter an infinite loop, as FSAs might have cycles. An infinite search space would also be an issue.
2. Check oldest states first (FIFO). Such a **breadth-first search** implementation could use a **queue**. The pitfalls here include an infinite search space (never terminates), and an ever-growing agenda might be outrageously memory-intensive.

For simple problems, DFS is typically preferred (for memory), but more complex methods such as **dynamic programming** or **A*** will prove useful for larger problems.

### Relating Deterministic and Non-Deterministic Automata

I've studied this in a course, and long story short, $L(\text{DFA})=L(\text{NFA})$. The proof's algorithm, as a reminder, is subset construction. Create a DFA whose states represent sets of states of the corresponding NFA. This can result in a DFA with as many as $2^N$ states.

## 2.3 Regular Languages and FSAs (p. 43)

Also, $L(\text{DFA})=L(\text{NFA})=L(\text{RE})$. We call these the **regular languages**. To formally define such langauges, we need three operations:

*Def*: Regular languages over alphabet $\mathbb{A}$ are defined as follows:
1. $\emptyset$ (the empty set) is a regular language.
2. $\forall a \in \{\mathbb{A} \cup \epsilon \}, \{ a \}$ is a regular language.
3. If $L_1$ and $L_2$ are regular langauges, then so are:
     - $L_1L_2 = \{ xy | x \in L_1,y \in L_2\}$, the **concatenation** of $L_1$ and $L_2$.
     - $L_1 \cup L_2$, the **union** of $L_1$ and $L_2$ (write $L_1|L_2$).
     - $L_1^*$, the **closure** of $L_1$.
     
Thus our earlier RegEx operators define all possible regular languages, excluding memory.

We can also prove that FSA have equivalent expressive power to regular expressions. First, consider some regular expression:
1. Take the atomic regular operands $\epsilon$, $\emptyset$, and $a$. We can easily define DFAs for these.
2. We'll induct on the operators of the regular expression:
    - Concatenation: Add an $\epsilon$-transition between the two FSAs representing the operands.
    - Union: Put a state on he beginning/end of the two operand's FSAs, connecting the operands to these new states using four total $\epsilon$-transitions.
    - Closure: Add an $\epsilon$-transition from the operand's end to its beginning and then one state on either end of the operand's FSA. Finally, add an $\epsilon$-transition from the first new state to the last.
    
As my own note, we can turn any $\epsilon$-FSA into a non-$\epsilon$ FSA using a particular algorithm this book doesn't cover. Same expressive power, don't worry.

## 2.4 Summary (p. 46)

In this chapter, we introduced a bunch of topics concerning regular expressions and finite state automata. I won't copy down this summary, see above or the pdf's page 46.

## Exercises (p.47)

I won't consider all of the exercises, but the following look interesting: 2.1.f, 2.4, 2.5, 2.6. I might return to these eventually but for now I'm RegEx'd out.

*--Exercises remain incomplete, last revised 5/29/2019--*